In [ ]:
# windows 也有这种坑爹的时候, 键盘的 shift 失灵. 可以打开屏幕键盘后解决

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, GenerationConfig
from transformers import BitsAndBytesConfig

c:\tech\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
quantization_config = BitsAndBytesConfig(
    False,
    True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

model_dir = r"G:\code\pretrain_model_dir\_modelscope\baichuan-inc\Baichuan2-13B-Chat"
tokenizer = AutoTokenizer.from_pretrained(
    model_dir, device_map="auto", trust_remote_code=True, torch_dtype=torch.float16
)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)
model.generation_config = GenerationConfig.from_pretrained(model_dir)

print(model.device, model.dtype)

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


bin c:\tech\Anaconda3\envs\nlp\lib\site-packages\bitsandbytes\libbitsandbytes_cuda121.dll


Loading checkpoint shards: 100%|██████████| 3/3 [00:18<00:00,  6.07s/it]

cuda:0 torch.float16


In [6]:
model.generation_config

GenerationConfig {
  "assistant_token_id": 196,
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 2048,
  "pad_token_id": 0,
  "repetition_penalty": 1.05,
  "temperature": 0.3,
  "top_k": 5,
  "top_p": 0.85,
  "user_token_id": 195
}

In [8]:
print(tokenizer.convert_ids_to_tokens(model.generation_config.assistant_token_id))
print(tokenizer.convert_ids_to_tokens(model.generation_config.user_token_id))

<reserved_107>
<reserved_106>


In [9]:
tokenizer.convert_ids_to_tokens([  195, 92955, 92342, 92955, 83546, 92333,  2081,   196])

['<reserved_106>', '讲', '一', '讲', '牛顿', '的', '发现', '<reserved_107>']

In [160]:
tokenizer.chat_template = """
{% for message in messages %}
{% if message['role'] == 'user' %}
{{ '<reserved_106>' + message['content'].strip() }}
{% elif message['role'] == 'system' %}
{{ message['content'].strip() }}
{% elif message['role'] == 'assistant' %}
{{ '<reserved_107>'  + message['content'] }}
{% endif %}
{% if loop.last and message['role'] != 'assistant' %}
{{ '<reserved_107>' }}
{% endif %}
{% endfor %}
""".strip().replace("\n", "")

In [158]:
# tokenizer.chat_template = """
# {% set ns = namespace(text='') %}
# {% for message in messages %}
# {% if message['role'] == 'user' %}
# {% set ns.text = ns.text + '<reserved_106>' + message['content'].strip() %}
# {% elif message['role'] == 'system' %}
# {% set ns.text = ns.text + message['content'].strip() %}
# {% elif message['role'] == 'assistant' %}
# {% set ns.text = ns.text + '<reserved_107>'  + message['content'] %}
# {% endif %}
# {% if loop.last and message['role'] != 'assistant' %}
# {% set ns.text = ns.text + '<reserved_107>' %}
# {% endif %}
# {% endfor %}

# {% if ns.text|length > 10 %} {# 如果 text 的长度超过 n #}
#   {{ ns.text[-10:] }} {# 那么输出 text 的最后 n 个字符 #}
# {% else %} {# 如果 text 的长度不超过 n #}
#   {{ ns.text }} {# 那么输出 text 的全部内容 #}
# {% endif %}
# """.strip().replace("\n", "")

In [22]:
tokenizer.chat_template

"{% for message in messages %}{% if message['role'] == 'user' %}{{ '<reserved_106>' + message['content'].strip() }}{% elif message['role'] == 'system' %}{{ message['content'].strip() }}{% elif message['role'] == 'assistant' %}{{ '<reserved_107>'  + message['content'] }}{% endif %}{% if loop.last and message['role'] != 'assistant' %}{{ '<reserved_107>' }}{% endif %}{% endfor %}"

# 感觉也不是很完善, 我还不知道怎么截断回答. jinja 本身模板应该是做不到这一点的, 因为还没有变成 token, 只是文本很难判断具体长度

应该从 `apply_chat_template` 方法入手

In [161]:
chat = [
    {"role": "user", "content": "讲一讲牛顿的发现"},
    {"role": "user", "content": "讲一讲牛顿的发现"},
]

tokenizer.apply_chat_template(chat, tokenize=False)

'<reserved_106>讲一讲牛顿的发现<reserved_106>讲一讲牛顿的发现<reserved_107>'

In [24]:
chat = [
    {"role": "assistant", "content": "讲一讲牛顿的发现"},
]

tokenizer.apply_chat_template(chat, tokenize=False)

'<reserved_107>讲一讲牛顿的发现'

In [25]:
pred = model.generate(**tokenizer("讲一讲牛顿的发现", return_tensors="pt").to('cuda:0'), max_new_tokens=512)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

讲一讲牛顿的发现



In [32]:
from transformers import pipeline, Conversation

# 这个长度不设置有点坑, 默认怎么会是 20, 都不知道是哪里设置的
model.config.max_length = 4096
tokenizer.pad_token_id = tokenizer.eos_token_id
chatbot = pipeline("conversational", model=model, tokenizer=tokenizer)
conversation = Conversation("讲一讲牛顿的发现")
conversation = chatbot(conversation)
conversation.generated_responses[-1]

Both `max_new_tokens` (=2048) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'艾萨克·牛顿（Isaac Newton）是英国的一位杰出的科学家、数学家和哲学家，他的发现和理论对科学革命产生了深远的影响。以下是牛顿的一些主要发现和成就：\n\n1. 万有引力定律：牛顿提出了万有引力定律，这是描述物体间引力的数学公式。根据这一定律，任何两个物体都会相互吸引，吸引力与它们的质量成正比，与它们之间的距离成反比。这一定律解释了地球和其他天体的运动，以及潮汐现象。\n\n2. 牛顿三定律：牛顿提出了三个描述物体运动的定律，即牛顿第一定律（惯性定律）、牛顿第二定律（动量定律）和牛顿第三定律（作用与反作用定律）。这些定律为经典力学奠定了基础，至今仍被广泛应用于物理学和工程学领域。\n\n3. 微积分：牛顿与德国数学家戈特弗里德·莱布尼茨（Gottfried Wilhelm Leibniz）独立发现了微积分。这是一种研究变化率和累积量的数学方法，对于物理学、工程学、经济学等多个领域具有重要意义。\n\n4. 光学：牛顿在光学领域的贡献也非常显著。他发现了白光实际上是由多种颜色的光组成的，通过实验证明了光的波动性。此外，他还发明了反射式望远镜，提高了观测天体的能力。\n\n5. 数学：牛顿在数学领域也有很多重要贡献，包括二项式定理、幂级数展开等。他的著作《自然哲学的数学原理》（Mathematical Principles of Natural Philosophy，简称《原理》）是科学史上最重要的著作之一，对科学革命产生了深远影响。\n\n总之，艾萨克·牛顿的发现和理论对科学和人类文明的发展产生了巨大影响。他的研究成果为现代科学技术的发展奠定了基础，被誉为科学史上最伟大的科学家之一。'

In [35]:
conversation.add_message({"role": "user", "content": "我刚刚提到了谁, 还有谁和他一样伟大, 举出三个例子"})

In [52]:
conversation = chatbot(conversation)
conversation.generated_responses[-1]

Both `max_new_tokens` (=2048) and `max_length`(=4096) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


'你提到的应该是艾萨克·牛顿，关于与他同样伟大的科学家，我可以为你列举三位：\n\n1. 阿尔伯特·爱因斯坦：他是20世纪最伟大的物理学家之一，提出了相对论，包括狭义相对论和广义相对论，对现代物理学产生了深远影响。他的质能方程E=mc²揭示了物质和能量之间的关系，对核能和粒子物理学的发展起到了关键作用。\n\n2. 尼古拉·特斯拉：他是19世纪末20世纪初的杰出科学家，对电气工程学和无线电通信做出了重大贡献。他发明了交流电系统，这一发明使得电力能够大规模远距离传输，极大地推动了现代社会的发展。\n\n3. 查尔斯·达尔文：他是生物进化论的奠基人，他的理论对生物学和生态学产生了深远影响。他的著作《物种起源》提出了自然选择的概念，解释了生物多样性的起源和演化过程。'

In [42]:
model.generation_config

GenerationConfig {
  "assistant_token_id": 196,
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 2048,
  "pad_token_id": 0,
  "repetition_penalty": 1.05,
  "temperature": 0.3,
  "top_k": 5,
  "top_p": 0.85,
  "user_token_id": 195
}